# Cell segmentation & classification with Random Forest Classifier (RFC)

**Content**  
1. *Cell segmentation*: Given a sparse user annotation of a single cell image, we can segment cells from the background using a pixel-wise Random Forest Classifier (RFC)  
2. *Cell classification*: Given a separated cell image and some sparse ground truth annotation we can classify cell objects based on size and shape measurements, e.g. derived using scikit-image `regionprops`. We will do this using a RFC as well.  

**Methodology**:
- This notebook has gaps (marked with a `#TODO`) that you have to fill. 
- This notebook only contains TODOs in the section (2.) about Cell Segmentation.
- If all gaps are filled in correctly, the notebook can be run.
- A solution notebook with all gaps filled is provided in the GitHub repository.

**Source, Inspiration**:  
This notebook is inspired by [Pixel Classification with Random Forest](https://haesleinhuepf.github.io/BioImageAnalysisNotebooks/20a_pixel_classification/scikit_learn_random_forest_pixel_classifier.html) and [Object Classification with Random Forest](https://haesleinhuepf.github.io/BioImageAnalysisNotebooks/27_cell_classification/sklearn_object_classification.html).

## 1. Imports and Data
We will use the following libraries and methods:

- `os`: For interacting with the operating system.
- `RandomForestClassifier` from `sklearn.ensemble`: The implementation of the RFC by scikit-learn.
- `io`, `filters` and `measure` from `skimage`: Read and write images, filters and other functionability.
- `numpy` (`np`): For numerical operations.
- `pandas` (`pd`): For tabular data manipulation and visualization.
- `matplotlib.pyplot` (`plt`): For data visualization.

In [ ]:
import os
from sklearn.ensemble import RandomForestClassifier
from skimage import io, filters, measure
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

We download all files that are not available (In the colab) from the internet (from the online GitHub repository of this workshop) and save them in the current workspace.

In [ ]:
import urllib.request  # helper library to download files

if not os.path.exists('data/blobs.tif'):
    os.makedirs('data', exist_ok=True)
    urllib.request.urlretrieve('https://github.com/moritzmarquardt/workshop_IBEC/raw/refs/heads/main/data/blobs.tif', 'data/blobs.tif')
    print('Loaded and saved image from remote file')

if not os.path.exists('data/blobs_segmentation_annotation.npy'):
    os.makedirs('data', exist_ok=True)
    urllib.request.urlretrieve('https://github.com/moritzmarquardt/workshop_IBEC/raw/refs/heads/main/data/blobs_segmentation_annotation.npy', 'data/blobs_segmentation_annotation.npy') 
    print('Loaded and saved annotations from remote file')

if not os.path.exists('data/blobs_classification_annotation.npy'):
    os.makedirs('data', exist_ok=True)
    urllib.request.urlretrieve('https://github.com/moritzmarquardt/workshop_IBEC/raw/refs/heads/main/data/blobs_classification_annotation.npy', 'data/blobs_classification_annotation.npy') 
    print('Loaded and saved class annotations from remote file')


In [ ]:
image = io.imread('data/blobs.tif')

plt.figure(figsize=(4, 4))
plt.imshow(image, cmap='gray')
plt.colorbar()
plt.title('Original example image')
plt.show()

print(f'Image shape: {image.shape}')
print(f'Image type: {image.dtype}')
print(f'Image max value: {image.max()}')
print(f'Image min value: {image.min()}')

## 2. Cell segmentation using RFC

### 2.1 (optional) Interactive manual annotation
**Important**: This is not possible in a Google Colab but only on a local python runtime. Therefore it is skipped in this tutorial, but the interested reader can try this out.

Using the two cells below, it is possible to annotate the image manually and save the annotations to a file to use them in the following.  
Feel free to test this out if you run this notebook on your PC.

In [ ]:
# import napari

# # start napari
# viewer = napari.Viewer()

# # add image
# viewer.add_image(image)

# # add an empty labels layer and keep it in a variable
# labels = viewer.add_labels(np.zeros(image.shape).astype(int))

After running the cell above and annotating manually in the napari window using the brush tool, execute the cell below to save the annotation.

In [ ]:
# annotations = labels.data

# np.save('data/blobs_manual_annotations.npy', annotations)

# napari.utils.nbscreenshot(viewer)

### 2.2 Load and visualize annotations

Here we load the annotations that were saved in the part above or the presaved example annotation is used.  
Annotations are the following:
- value 1 is background
- value 2 is a cell

In [ ]:
annotations = np.load('data/blobs_segmentation_annotation.npy')

# visualize the manual annotations
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax = ax.ravel()
im0 = ax[0].imshow(annotations)
ax[0].set_title('Manual Annotations')
cbar = plt.colorbar(im0, ax=ax[0], orientation='vertical')
cbar.set_ticks([0, 1, 2])
cbar.set_ticklabels([0, 1, 2])

im1 = ax[1].imshow(image, cmap='gray')
ax[1].imshow(annotations, alpha=0.5)
ax[1].set_title('Overlay of Image and Annotations')

plt.show()

### 2.3 Feature extraction
In this subsection, we generate features for each pixel from the original image since we will be classifying pixels in the end.
The idea is to have different values (features) for each pixel to make the classification stronger.  
These values will be obtained by applying different filters to the image.

**TODO**:
1. Implement the two filters from skimage.filters to generate features

In [ ]:

# define additional features

#TODO: Use the filters.gaussian function from the skimage module with sigma=2
blurred_image = 

#TODO Use the filters.sobel function from the skimage module
edges_image = 

# The ravel() function turns a nD image into a 1-D image.
features = [
    image.ravel(),
    blurred_image.ravel(),
    edges_image.ravel()
]

# Transpose the features to have the shape (n_samples, n_features)
features = np.asarray(features).T

# visualize the features as a pandas dataframe
df_features = pd.DataFrame(features)
df_features.columns = ['original', 'blurred', 'edges']
df_features

In [ ]:

# visualise feature images
fig, axes = plt.subplots(1, 3, figsize=(12,4))

# reshape(image.shape) is the opposite of ravel() here. We just need it for visualization.
axes[0].imshow(df_features["original"].values.reshape(image.shape), cmap='gray')
axes[0].set_title('Original')
axes[1].imshow(df_features["blurred"].values.reshape(image.shape), cmap='gray')
axes[1].set_title('Blurred using Gaussian filter')
axes[2].imshow(df_features["edges"].values.reshape(image.shape), cmap='gray')
axes[2].set_title('Edges using Sobel filter')
plt.show()

#TODO (for fast people) Play around with the sigma parameter of the gaussian blur filter. A larger sigma corresponds to a larger gaussian kernel and thus more blurring. Find a balance so that noise is reduced but edges are still visible.

### 2.4 Format training data
The RandomForestClassifier expects an X array of shape (n_samples, n_features) and a y array of shape (n_features).

In [ ]:
X_train = features
y_train = annotations.ravel()

# use only annotated pixels for training
mask = y_train > 0
X_train = X_train[mask]
y_train = y_train[mask]

print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)

# visualize the training data
df_train_x_y = pd.DataFrame(X_train)
df_train_x_y['label'] = y_train
# rename columns to feature_i
df_train_x_y.columns = [f'feature_{i}' for i in range(3)] + ['label']
df_train_x_y.head()

### 2.5 Train the Random Forest Classifier

**TODO**:
1. Use the scikit-learn's RandomForestClassifier to create and fit the Classifier.

In [ ]:
# train test split using df_train_x_y
from sklearn.model_selection import train_test_split
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0) 
print(X_train_split.shape, y_train_split.shape)
print(X_val.shape, y_val.shape)


#TODO: initialize a RandomForestClassifier with a max_depth of 2 and a random_state of 0. 
classifier = 

#TODO: Train the classifier with the training data X_train_split and y_train_split (use the fit method of the classifier)


#evaluate the classifier
print(f'Validation accuracy: {classifier.score(X_val, y_val):.2f}')


In [ ]:
# visualise one of the trees of the "random forest"
from sklearn.tree import plot_tree
plt.figure(figsize=(6, 4))
plot_tree(classifier.estimators_[0], filled=True)
plt.show()

### 2.6 Use the classifier for segmentation

**TODO**:
1. Use the prediciton method of the classifier to predict all pixels of the image and get a segmented image

In [ ]:
X_predict = features # use all features for all pixels to predict

#TODO Use the predict function of the classifier to predict the annotations (cell or background) for the prediction data X_predict
res = 

res = res - 1 # we subtract 1 to make background = 0 (this is necessary, since the background is annotated with 1 in the manual annotations)

# visualize the results
res_image = res.reshape(image.shape) # reshape the 1D result array back to the 2D image shape
fig, ax = plt.subplots(1, 3, figsize=(12, 4))
ax = ax.ravel()
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original image')

# Display predicted segmentation without specifying cmap (default)
segmentation_plot = ax[1].imshow(res_image)
ax[1].set_title('Predicted segmentation')

# Extract colormap colors
cmap = segmentation_plot.get_cmap()
norm = segmentation_plot.norm  # Normalization of values (0 to 1)
background_color = cmap(norm(0))  # Color for background (0)
segment_color = cmap(norm(1))  # Color for segment (1)

# Add a legend with the correct colors
legend_labels = ['Background (0)', 'Segmented cell (1)']
colors = [background_color, segment_color]  # Extracted colors
patches = [plt.Line2D([0], [0], color=color, marker='o', linestyle='None', markersize=10, label=label)
           for color, label in zip(colors, legend_labels)]
ax[1].legend(handles=patches, loc='upper right')

ax[2].imshow(image, cmap='gray', alpha=0.8)
ax[2].contour(res_image == 1, colors='r')
ax[2].set_title('Overlay of image and segment contours')
## add legend for contours
patches = [plt.Line2D([0], [0], color='r', marker='o', linestyle='None', markersize=10, label='Segment contours')]
ax[2].legend(handles=patches, loc='upper right')

plt.show()

#TODO (for fast people) Play around with the max_depth parameter to see how it affects the classification performance. A larger max_depth corresponds to a more complex model that can capture more details in the data. However, it can also lead to overfitting. Find a balance that works well for this data.

### 2.7 Labeling the segmented objects
In this step, each connected component in the segmented image is assigned a unique label. This process is essential for the next phase, where we classify individual objects based on their properties.

**TODO**: 
1. label the segmented objects

In [ ]:
#TODO Use the measure.label function from the skimage module to label the objects in the segmentation res_image
labels = 

plt.figure(figsize=(4, 4))
plt.imshow(labels)
cbar = plt.colorbar()
plt.title('Labeled objects')
plt.show()

### Compare with non ml methods

- Otsu thresholding  

In [ ]:
# otsu thresholding
thresh = filters.threshold_otsu(image)
binary = image > thresh
fig, ax = plt.subplots(1, 3, figsize=(10, 10))
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original image')
ax[1].imshow(res_image)
ax[1].set_title('RFC Segmentation')
ax[2].imshow(binary)
ax[2].set_title('Otsu thresholding')
plt.show()

Otsu thresholding works well here since the example image is greyscale, but these classical methods get to their limits when images are colourful and more complex.

## 3. Classification using RFC

Now we will train a second Random Forest classifier (RFC) to classify each segmented and labeled object based on its shape properties.


### 3.1 Load annotations
Similar to the segmentation, we need an annotation of the image which can be done manually by using for example the napari plugin similar to the segmentation part. Here we load an annotation that has already been done.  

Annotations are the following:
- value 1: large cells
- value 2: elongated cells
- value 3: small cells

In [ ]:
# annotation with classes:
# 1 (blue): large cells
# 2 (green): elongated cells
# 3 (yellow): small cells
class_annotation = np.load('data/blobs_classification_annotation.npy')

# visualize
fig, ax = plt.subplots(1,3, figsize=(15,15))
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Original image')
ax[1].imshow(labels)
ax[1].set_title('RFC Segmentation')
# Display the annotation image with transparency
ax[2].imshow(image, cmap='gray')
annotation_plot = ax[2].imshow(class_annotation, alpha=0.8)
ax[2].set_title('Manual annotations')

# Extract colormap and normalization from the annotation plot
cmap = annotation_plot.get_cmap()
norm = annotation_plot.norm

# Define the colors and labels for the legend
legend_labels = ['Large cells (1)', 'Elongated cells (2)', 'Small cells (3)']
colors = [cmap(norm(1)), cmap(norm(2)), cmap(norm(3))]

# Create legend patches
patches = [plt.Line2D([0], [0], color=color, marker='o', linestyle='None', markersize=10, label=label)
           for color, label in zip(colors, legend_labels)]

# Add legend to the lower left corner
ax[2].legend(handles=patches, loc='lower left')
plt.show()

### 3.2 Feature extraction
The first step to classify objects according to their properties is feature extraction. This is analog to building the feature stack in the segmentation part. The difference here is, we do not extract features for each pixel, but for each segmented object (connected labeled pixels) since we will be classifying the objects in the end.

In [ ]:
stats = measure.regionprops(labels, intensity_image=image)

# read out specific measurements as features
label_ids =          np.asarray([s.label for s in stats])
areas =              np.asarray([s.area for s in stats])
minor_axis_lengths = np.asarray([s.minor_axis_length for s in stats])
major_axis_lengths = np.asarray([s.major_axis_length for s in stats])

# compute additional parameters
aspect_ratios = major_axis_lengths / minor_axis_lengths

We also read out the maximum intensity of every labeled object from the ground truth annotation. These values will serve to train the classifier.

In [ ]:
annotation_stats = measure.regionprops(labels, intensity_image=class_annotation)

annotated_class = np.asarray([s.intensity_max for s in annotation_stats])

### 3.3 Format data

To look at the data before it is fed to the training, we visualize it as [pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). Note: The rows with `annotated_class=0` correspond to labels that have not been annotated.

In [ ]:
data = {
    'label': label_ids,
    'area': areas,
    'minor_axis': minor_axis_lengths,
    'major_axis': major_axis_lengths,
    'aspect_ratio': aspect_ratios,
    'annotated_class': annotated_class
}

table = pd.DataFrame(data)

From that table, we extract now a table that only contains the annotated rows/labels.

In [ ]:
annotated_table = table[table['annotated_class'] > 0]

As we do not want to use all columns for training, we now select the right columns. It is recommended to write a short convenience function `select_data` for this, because we will reuse it later for prediction.

In [ ]:
def select_data(table):
    return np.asarray([
        table['area'],
        table['aspect_ratio']
    ])

training_data = select_data(annotated_table).T
ground_truth = annotated_table['annotated_class'].tolist()

### 3.4 Train the Random Forest classifier

Next, we can train the [Random Forest Classifer](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). It needs training data and ground truth in the format presented above.

In [ ]:
classifier_cells = RandomForestClassifier(max_depth=2, n_estimators=10, random_state=0)
classifier_cells.fit(training_data, ground_truth)

### 3.5 Predict the ojects classes

To apply a classifier to the whole dataset, or any other dataset, we need to bring the data into the same format as used for training. We can reuse the function `select_data` for that. Furthermore, we need to drop rows from our table where not-a-number values appeared.

In [ ]:
table_without_nans = table.dropna(how="any")

all_data = select_data(table_without_nans).T
all_data
# where is alldata inf
np.where(np.isinf(all_data))
all_data[all_data == np.inf] = np.nan

We can then hand over `all_data` to the classifier for prediction.

In [ ]:
table_without_nans['predicted_class'] = classifier_cells.predict(all_data)

We can then merge the table containing the `predicted_class` column with the original table. In the resulting `table_with_prediction`, we still need to decide how to handle `NaN` values. It is not possible to classify those because measurements are missing. Thus, we replace the class of those with 0 using [`fillna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html).

In [ ]:
# merge prediction with original table
table_with_prediction = table.merge(table_without_nans, how='outer', on='label')
# replace not predicted (NaN) with 0
table_with_prediction['predicted_class'] = table_with_prediction['predicted_class'].fillna(0)

From that table, we can extract the column containing the prediction and use [`replace_intensities`](https://clij.github.io/clij2-docs/reference_replaceIntensities) to generate a `class_image`. The background and objects with NaNs in measurements will have value 0 in that image.

In [ ]:
# we add a 0 for the class of background at the beginning
predicted_class = [0] + table_with_prediction['predicted_class'].tolist() 

In [ ]:
#class_image = replace_intensities(labels, predicted_class)
class_image = labels.copy()
for s in stats:
    class_image[class_image == s.label] = predicted_class[s.label]

plt.imshow(class_image)
cbar = plt.colorbar()
cbar.set_ticks([0, 1, 2, 3])
cbar.set_ticklabels([0, 1, 2, 3])
plt.title('Predicted classes')
plt.show()